In [ ]:
%reload_ext autoreload
%autoreload 2
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import math

import pytorch_lightning as pl
import torch
from munch import Munch
from torch.utils.data import DataLoader, random_split

from qsr_learning.data import DRLDataset
from qsr_learning.models import DRLNet

In [ ]:
config = Munch()

# Dataset

In [ ]:
from qsr_learning.entity import emoji_names

In [ ]:
config.dataset = Munch(
    entity_names=["octopus", "trophy", "frog", "ghost"],
    excluded_combinations=[],
    relation_names=["left_of", "right_of"],
    num_entities=2,
    frame_of_reference="intrinsic",  # absolute
    w_range=(32, 32),
    h_range=(32, 32),
    theta_range=(0, 2 * math.pi),
    add_bbox=False,
    add_front=False,
    transform=None,
    canvas_size=(224, 224),
    num_samples=10 ** 5,
    root_seed=0,
)

In [ ]:
train_dataset = DRLDataset(
    **{
        **config.dataset,
        **dict(
            entity_names=["octopus", "trophy", "frog", "ghost"],
            excluded_combinations=[("octopus", "trophy")],
            num_samples=10 ** 5,
            root_seed=0,
        ),
    }
)

In [ ]:
validation_dataset = DRLDataset(
    **{
        **config.dataset,
        **dict(
            entity_names=["octopus", "trophy"],
            excluded_combinations=[],
            num_samples=10 ** 4,
            root_seed=train_dataset.num_samples,
        ),
    }
)

# Data Loader

In [ ]:
config.data_loader = Munch(
    batch_size=128,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)

In [ ]:
train_loader = DataLoader(train_dataset, **config.data_loader)
validation_loader = DataLoader(
    validation_dataset, **{**config.data_loader, "shuffle": False}
)

In [ ]:
# from tqdm.auto import trange

# for i in trange(len(train_dataset)):
#     train_dataset[i]

# Model

In [ ]:
config.model = Munch(
    vision_model="resnet18",
    image_size=(3, *config.dataset.canvas_size),
    num_embeddings=len(train_dataset.word2idx),
    embedding_dim=10,
    question_len=train_dataset[0][1].shape.numel(),
)

In [ ]:
model = DRLNet(**config.model)

# Trainer 

In [ ]:
config.trainer = Munch(
    gpus=1,
    max_epochs=100,
    precision=32,
    limit_train_batches=1.0,
    limit_val_batches=1.0,
)

In [ ]:
trainer = pl.Trainer(**config.trainer)
trainer.fit(model, train_loader, validation_loader)